In [13]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.linear_model import LinearRegression
#parse data 
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
#label encoding on categorical data 

#FAMA 49CRSP Common Stocks 
df = pd.read_csv('ee6d2f60cdafb550.csv')
df = df.dropna()

In [28]:
import sklearn.preprocessing 
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.model_selection import train_test_split

#preprocessing here
#sort by date 
df = df.sort_values(by = 'public_date', ascending = True)

#encode integer categories into numbers 
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(df.FFI49_desc)
print(integer_encoded)
df.FFI49_desc = integer_encoded
df.divyield_Median = [float(x.strip('%'))/100 for x in df.divyield_Median]
print(df)

#todo: https://www.gsb.stanford.edu/library/articles/databases/links/financial-ratios-suite?fbclid=IwAR0EGNGk9DdxQjEHfdaoUhdY3tNzAWDogYDzuuJi1zT_muL-uJtWQw19Fzk

#get output first 
ewlabels = df.indret_ew
vwlabels = df.indret_vw

#3year on year change as a prediction feature, raw pct change 
yoythree = ewlabels.diff(periods = 3)
#3 years rolling percent change, averaged ie. (y1-y2 + (y3-y2)change)/2 
rollavgpct = ewlabels.rolling(3).mean()

#drop first 3 years
df = df.iloc[3:]
ewlabels = ewlabels.iloc[3:]
yoythree = yoythree.iloc[3:]
#yoypctthree = yoypctthree.iloc[3:]
rollavgpct = rollavgpct.iloc[3:]

#add -1 and 1 so the bins will take on bins to be equal and set to max -1 and 1
extrema = pd.Series([-1,1])
ewnlabels = ewlabels.append(extrema)

#make a new output (bucket by percentage?)
enc = KBinsDiscretizer(n_bins=8, encode='ordinal',strategy = 'uniform')
ewnlabels = np.asarray(ewnlabels)
ewnlabels = ewnlabels.reshape((-1,1))
labels_binned = enc.fit_transform(ewnlabels)

labels_binned = labels_binned[:-2]

#1 Split-Timer series data, 0.64 Train, 0.16 dev, 0.2 Test
x_train, x_test, y_train, y_test = train_test_split(df, labels_binned, test_size = 0.2, shuffle = False)
x_tra, x_dev, y_tra, y_dev = train_test_split(x_train, y_train, test_size = 0.2, shuffle = False)

[25 25 25 ... 27 45 47]


AttributeError: 'float' object has no attribute 'strip'

In [ ]:
print(x_train)

In [39]:
#tutorial keras practice
#https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/

from keras.models import Sequential
from keras.layers import Dense, Activation,Softmax
from keras.optimizers import SGD
from sklearn.metrics import mean_squared_error
import numpy

model = Sequential()
#parameters = number of neurons, initialization method, activation function
model.add(Dense(32, input_dim=76, init = 'uniform', activation = 'relu'))
model.add(Dense(16, init = 'uniform', activation = 'relu'))
model.add(Dense(1, init = 'uniform', activation = 'sigmoid'))

# For a binary classification problem
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=25, batch_size=32)


print("----------------------------------------------------------")
scores = model.evaluate(x_train,y_train)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")

y_devpred = model.predict(x_dev)
print("--------------------------------------------")
print(mean_squared_error(y_dev,y_devpred))

C:\Users\carol\Anaconda3\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(32, input_dim=76, activation="relu", kernel_initializer="uniform")`
  if sys.path[0] == '':
C:\Users\carol\Anaconda3\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(16, activation="relu", kernel_initializer="uniform")`
  del sys.path[0]
C:\Users\carol\Anaconda3\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="uniform")`
  


Epoch 1/25
7435/7435 [==============================] - 1s 139us/step - loss: 57.7368 - acc: 0.0000e+00
Epoch 2/25
7435/7435 [==============================] - 0s 63us/step - loss: 57.7368 - acc: 0.0000e+00
Epoch 3/25
7435/7435 [==============================] - 0s 46us/step - loss: 57.7368 - acc: 0.0000e+00
Epoch 4/25
7435/7435 [==============================] - 0s 50us/step - loss: 57.7368 - acc: 0.0000e+00
Epoch 5/25
7435/7435 [==============================] - 0s 48us/step - loss: 57.7368 - acc: 0.0000e+00
Epoch 6/25
7435/7435 [==============================] - 0s 50us/step - loss: 57.7368 - acc: 0.0000e+00
Epoch 7/25
7435/7435 [==============================] - 0s 48us/step - loss: 57.7368 - acc: 0.0000e+00
Epoch 8/25
7435/7435 [==============================] - 0s 56us/step - loss: 57.7368 - acc: 0.0000e+00
Epoch 9/25
7435/7435 [==============================] - 0s 53us/step - loss: 57.7368 - acc: 0.0000e+00
Epoch 10/25
7435/7435 [==============================] - 0s 58us/step - 

In [51]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD
from sklearn.metrics import mean_squared_error
import numpy

model = Sequential()
model.add(Dense(32, input_dim=76, init = 'uniform', activation = 'relu'))
model.add(Dense(16, init = 'uniform', activation = 'relu'))
model.add(Dense(8, init = 'uniform', activation = 'softmax'))

# For a multi-class classification problem
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x_train, y_train, epochs=25, batch_size=32)


print("----------------------------------------------------------")
scores = model.evaluate(x_train,y_train)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
y_devpred = model.predict(x_dev)
print("--------------------------------------------")
print(mean_squared_error(y_dev,y_devpred))

C:\Users\carol\Anaconda3\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(32, input_dim=76, activation="relu", kernel_initializer="uniform")`
  
C:\Users\carol\Anaconda3\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(16, activation="relu", kernel_initializer="uniform")`
  if __name__ == '__main__':
C:\Users\carol\Anaconda3\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(8, activation="softmax", kernel_initializer="uniform")`
  # Remove the CWD from sys.path while we load stuff.


ValueError: Error when checking target: expected dense_79 to have shape (8,) but got array with shape (1,)

In [50]:
from keras.models import Sequential
from keras.layers import Dense, Activation,Softmax
from keras.optimizers import SGD

model = Sequential()
model.add(Dense(32, input_shape = (x_train.shape)))
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))

# For a multi-class classification problem
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# For a binary classification problem
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# For a mean squared error regression problem
model.compile(optimizer='rmsprop',
              loss='mse')

model.fit(x_train, y_train, epochs=5, batch_size=32)
y_devpred = model.predict(x_dev)
print("--------------------------------------------")
print(mean_squared_error(y_dev,y_devpred))


ValueError: Error when checking input: expected dense_74_input to have 3 dimensions, but got array with shape (7435, 76)